In [5]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [6]:
# Carregar os dados
metric_df = pd.read_pickle("../data/ts.pkl")

# Resample para intervalos de 30 minutos
ts = metric_df["value"].astype(float).resample("30min").mean()

# Dividir em treino e teste
train = ts[:"2021-02-07"]
test = ts["2021-02-08":]

# Escalonar os dados
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
test_scaled = scaler.transform(test.values.reshape(-1, 1))

# Definir número de passos e features
n_steps = 40
n_features = 1

# Criar sequências para treino
X_train, y_train = [], []
for i in range(n_steps, len(train_scaled)):
    X_train.append(train_scaled[i-n_steps:i, 0])
    y_train.append(train_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape para 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], n_features))

# Criar sequências para teste
X_test, y_test = [], []
for j in range(n_steps, len(test_scaled)):
    X_test.append(test_scaled[j-n_steps:j, 0])
    y_test.append(test_scaled[j, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape para 3D
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], n_features))

In [7]:
# Definir a URI do MLflow local
mlflow.set_tracking_uri("http://localhost:5001")

In [8]:
# Função Objetivo que treina o modelo LSTM e retorna RMSE e R-squared
def objective_function(params):
    """
    Função objetivo para o MRFO.
    params: array-like, contendo os hiperparâmetros [lstm_units, dropout_rate, batch_size, learning_rate, n_steps, epochs]
    Retorna: RMSE do modelo treinado com os hiperparâmetros especificados.
    """
    # Garantir que lstm_units seja um valor inteiro e dentro dos limites
    lstm_units = max(50, min(150, int(params[0])))  # Limitar entre 50 e 150
    
    # Garantir que dropout_rate esteja entre 0 e 1
    dropout_rate = max(0, min(1, params[1]))  # Limitar entre 0 e 1
    
    # Garantir que batch_size seja um valor inteiro positivo adequado
    batch_size = max(16, min(128, int(params[2])))  # Garantir que seja pelo menos 16 e no máximo 128
    
    # Garantir que learning_rate esteja dentro dos limites
    learning_rate = max(0.0001, min(0.001, params[3]))  # Limitar a learning_rate entre 0.0001 e 0.001
    
    # Garantir que n_steps seja adequado e não maior que o conjunto de dados
    n_steps = max(10, min(min(len(train_scaled), len(test_scaled)) - 1, int(params[4])))  # Limitar o número de passos temporais entre 10 e o tamanho dos dados menos 1
    
    # Garantir que o número de épocas seja adequado
    epochs = max(10, min(200, int(params[5])))  # Limitar o número de épocas entre 10 e 200

    # Criar sequências de input data para treino
    X_train, y_train = [], []
    for i in range(n_steps, len(train_scaled)):
        X_train.append(train_scaled[i-n_steps:i, 0])
        y_train.append(train_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Reshape para 3D
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Criar sequências de input data para teste
    X_test, y_test = [], []
    for j in range(n_steps, len(test_scaled)):
        X_test.append(test_scaled[j-n_steps:j, 0])
        y_test.append(test_scaled[j, 0])
    X_test, y_test = np.array(X_test), np.array(y_test)

    # Reshape para 3D, desde que o X_test tenha dados suficientes
    if X_test.shape[0] > 0:
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    else:
        return float("inf"), float("inf"), float("inf"), float("inf")  # Retornar valor alto para indicar erro

    # Iniciar um experimento no MLflow para cada execução
    with mlflow.start_run():
        # Registrar os hiperparâmetros no MLflow
        mlflow.log_param("lstm_units", lstm_units)
        mlflow.log_param("dropout_rate", dropout_rate)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("n_steps", n_steps)
        mlflow.log_param("epochs", epochs)

        # Definir o modelo com os hiperparâmetros especificados
        model = Sequential()
        model.add(LSTM(lstm_units, activation='tanh', input_shape=(n_steps, 1), return_sequences=True))
        model.add(Dropout(dropout_rate))
        model.add(LSTM(lstm_units, activation='tanh'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(1))

        # Compilar o modelo com o otimizador Adam e a taxa de aprendizado especificada
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mse')

        # Treinar o modelo
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Fazer previsões e calcular o RMSE, MSE, R-squared e MAE
        test_pred = model.predict(X_test)
        test_pred = scaler.inverse_transform(test_pred)
        rmse = np.sqrt(mean_squared_error(test.values[n_steps:], test_pred))
        mse = mean_squared_error(test.values[n_steps:], test_pred)
        r2 = r2_score(test.values[n_steps:], test_pred)
        mae = mean_absolute_error(test.values[n_steps:], test_pred)

        # Registrar as métricas no MLflow
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        # Salvar o modelo no MLflow
        mlflow.keras.log_model(model, "model")

    return rmse, mse, r2, mae

# Implementação do Algoritmo Manta Ray Foraging Optimization (MRFO)
class MRFO:
    def __init__(self, obj_func, dim, SearchAgents_no, max_iter, lb, ub):
        self.obj_func = obj_func  # Função objetivo a ser minimizada
        self.dim = dim  # Dimensionalidade do problema (número de hiperparâmetros)
        self.SearchAgents_no = SearchAgents_no  # Número de agentes (manta rays)
        self.max_iter = max_iter  # Número máximo de iterações
        self.lb = lb  # Limites inferiores dos parâmetros
        self.ub = ub  # Limites superiores dos parâmetros

        # Inicialização das posições dos agentes
        self.positions = np.random.uniform(0, 1, (self.SearchAgents_no, self.dim)) * (self.ub - self.lb) + self.lb
        self.fitness = np.array([self.obj_func(self.clip_params(ind))[0] for ind in self.positions])

        # Encontrar a melhor posição inicial
        self.best_idx = np.argmin(self.fitness)
        self.gbest = self.positions[self.best_idx].copy()
        self.gbest_fitness = self.fitness[self.best_idx]

    def clip_params(self, params):
        """
        Função auxiliar para garantir que os parâmetros fiquem dentro dos limites definidos.
        """
        params[0] = max(50, min(150, params[0]))  # LSTM units
        params[1] = max(0, min(1, params[1]))  # Dropout rate
        params[2] = max(16, min(128, params[2]))  # Batch size
        params[3] = max(0.0001, min(0.001, params[3]))  # Learning rate
        params[4] = max(10, min(min(len(train_scaled), len(test_scaled)) - 1, params[4]))  # n_steps
        params[5] = max(10, min(200, params[5]))  # epochs
        return params

    def chain_foraging(self, i):
        """Implementação da fase de Chain Foraging"""
        r = np.random.rand(self.dim)
        self.positions[i] = self.positions[i] + r * (self.gbest - self.positions[i])
        self.positions[i] = self.clip_params(self.positions[i])

    def cyclone_foraging(self, i, t, max_iter):
        """Implementação da fase de Cyclone Foraging"""
        r = np.random.rand(self.dim)
        A = 2 * (1 - t / max_iter)  # Fator de controle que diminui ao longo das iterações
        direction = np.random.choice([-1, 1], size=self.dim)  # Escolher aleatoriamente o sentido (horário/anti-horário)
        self.positions[i] = self.positions[i] + A * direction * r * (self.gbest - self.positions[i])
        self.positions[i] = self.clip_params(self.positions[i])

    def somersault_foraging(self, i):
        """Implementação da fase de Somersault Foraging"""
        S = 2 * np.random.rand(self.dim) - 1  # Vetor aleatório entre -1 e 1
        somersault_factor = 2  # Fator que controla a magnitude do somersault
        self.positions[i] = self.positions[i] + somersault_factor * (S * self.gbest - self.positions[i])
        self.positions[i] = self.clip_params(self.positions[i])

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.SearchAgents_no):
                # Escolher aleatoriamente uma fase de forrageamento a ser aplicada
                if np.random.rand() < 0.5:
                    self.chain_foraging(i)
                else:
                    self.cyclone_foraging(i, t, self.max_iter)

                # Avaliar a nova posição
                fitness_candidate = self.obj_func(self.clip_params(self.positions[i]))[0]

                # Atualizar a posição e a melhor solução encontrada
                if fitness_candidate < self.fitness[i]:
                    self.fitness[i] = fitness_candidate
                    if fitness_candidate < self.gbest_fitness:
                        self.gbest_fitness = fitness_candidate
                        self.gbest = self.positions[i].copy()

            # Somersault Foraging aplicado a todos os agentes uma vez por iteração
            for i in range(self.SearchAgents_no):
                self.somersault_foraging(i)

            print(f"Iteração {t+1}/{self.max_iter}, Melhor Fitness (RMSE): {self.gbest_fitness:.4f}")

        return self.gbest, self.gbest_fitness

# Definir os limites dos hiperparâmetros
dim = 6  # Agora com 6 hiperparâmetros (lstm_units, dropout_rate, batch_size, learning_rate, n_steps, epochs)
SearchAgents_no = 10  # Número de agentes (mantas)
max_iter = 20  # Número máximo de iterações

# Limites para cada hiperparâmetro
lb = np.array([50, 0.2, 32, 0.0001, 10, 10])  # Limites inferiores
ub = np.array([150, 0.5, 128, 0.001, 100, 200])  # Limites superiores

# Inicializar o algoritmo MRFO
mrfo = MRFO(objective_function, dim, SearchAgents_no, max_iter, lb, ub)

# Executar a otimização
best_params, best_fitness = mrfo.optimize()

# Obter as métricas para o melhor conjunto de hiperparâmetros
best_rmse, best_mse, best_r2, best_mae = objective_function(best_params)

# Imprimir os melhores hiperparâmetros com base no RMSE
print(f"\nMelhores Hiperparâmetros (baseado no RMSE):")
print(f"LSTM Units: {int(best_params[0])}")
print(f"Dropout Rate: {best_params[1]:.2f}")
print(f"Batch Size: {int(best_params[2])}")
print(f"Learning Rate: {best_params[3]:.6f}")
print(f"n_steps: {int(best_params[4])}")
print(f"Épocas: {int(best_params[5])}")

# Imprimir as métricas para o melhor conjunto de hiperparâmetros
print(f"\nMétricas para o Melhor Conjunto de Hiperparâmetros:")
print(f"RMSE: {best_rmse:.4f}")
print(f"MSE: {best_mse:.4f}")
print(f"R-squared: {best_r2:.4f}")
print(f"MAE: {best_mae:.4f}")


/Users/wrosalem/Documents/0x0-Coding/GitHub/UNESP-Com_Inspirada-Artigo/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


2024/10/02 00:51:18 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/02 00:51:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/02 00:51:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run sneaky-lamb-406 at: http://localhost:5001/#/experiments/0/runs/6818506e66c040ac9c2352aa399150cf.
2024/10/02 00:51:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/0.
/Users/wrosalem/Documents/0x0-Coding/GitHub/UNESP-Com_Inspirada-Artigo/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step


2024/10/02 00:51:30 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/02 00:51:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/02 00:51:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run omniscient-lynx-556 at: http://localhost:5001/#/experiments/0/runs/c3754e0210bb43d6a8a498459b778779.
2024/10/02 00:51:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/0.
/Users/wrosalem/Documents/0x0-Coding/GitHub/UNESP-Com_Inspirada-Artigo/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


2024/10/02 02:17:09 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/02 02:17:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/02 02:17:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run nosy-gnat-641 at: http://localhost:5001/#/experiments/0/runs/e50b01aee1b043f69eb2e7834658fe10.
2024/10/02 02:17:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/0.
/Users/wrosalem/Documents/0x0-Coding/GitHub/UNESP-Com_Inspirada-Artigo/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteração 20/20, Melhor Fitness (RMSE): 17032842.3999
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


2024/10/02 02:17:37 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/02 02:17:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/02 02:17:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run invincible-croc-344 at: http://localhost:5001/#/experiments/0/runs/bf10611b14184fc2b667de1a715c0dfc.
2024/10/02 02:17:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/0.



Melhores Hiperparâmetros (baseado no RMSE):
LSTM Units: 118
Dropout Rate: 0.00
Batch Size: 16
Learning Rate: 0.000993
n_steps: 10
Épocas: 121

Métricas para o Melhor Conjunto de Hiperparâmetros:
RMSE: 20363099.4451
MSE: 414655819009341.3125
R-squared: 0.9070
MAE: 10966906.7556
